In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import classifier

In [2]:
path = requests.get('https://inshorts.com/en/read')

In [13]:
sia = SentimentIntensityAnalyzer()

In [48]:
data_set = pd.DataFrame(columns = ['Date', 'Category', 'Polarity', 'Sentiment'])

In [3]:
path

<Response [200]>

In [4]:
path.status_code

200

In [5]:
page = path.content

In [6]:
soup = BeautifulSoup(page, 'html.parser')

In [7]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE doctype html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>
    /* The Modal (background) */
    .modal_contact {
        display: none; /* Hidden by default */
        position: fixed; /* Stay in place */
        z-index: 8; /* Sit on top */
        left: 0;
        top: 0;
        width: 100%; /* Full width */
        height: 100%;
        overflow: auto; /* Enable scroll if needed */
        background-color: rgb(0,0,0); /* Fallback color */
        background-color: rgba(0,0,0,0.4); /* Black w/ opacity */
    }

    /* Modal Content/Box */
    .modal-content {
        background-color: #fefefe;
        margin: 15% auto;
        padding: 20px !important;
        padding-top: 0 !important;
        /* border: 1px solid #888; */
        text-align: center;
        position: relative;
        border-radius: 6px;
    }

    /* The Close Button */
    .close {
      left: 90%;
      color: #aaa;
      float: right;
      font-size: 28px;


In [8]:
card_stack = soup.find(class_='card-stack')

In [9]:
articles = card_stack.find_all(class_="news-card z-depth-1")

In [49]:
for article in articles:
    heading = article.find('span',{'itemprop':'headline'}).get_text()
    article_content = article.find('div',{'itemprop':'articleBody'}).get_text()
    datePublished = article.find('span',{'class':'date'}).get_text()
    print(heading)
    category = classifier.predict_category(article_content)
    polarity = sia.polarity_scores(article_content)['compound']
    sentiment = 'Positive' if polarity > 0 else 'Negative' if polarity < 0 else 'Neutral'
    result = {'Date':datePublished, 'Category':category, 'Polarity':polarity, 'Sentiment':sentiment}
    data_set = data_set.append(result, ignore_index=True)
    print(result)
    print()

Uddhav Thackeray takes oath as Maharashtra CM
{'Date': '28 Nov', 'Category': 'politics', 'Polarity': 0.0, 'Sentiment': 'Neutral'}

I'm unable to attend oath-taking, wish you the best: Sonia to Uddhav
{'Date': '28 Nov', 'Category': 'business', 'Polarity': -0.4404, 'Sentiment': 'Negative'}

Commando 3 trailer shows villain lifting up school girl's skirt; criticised
{'Date': '28 Nov', 'Category': 'entertainment', 'Polarity': -0.8442, 'Sentiment': 'Negative'}

World's most valuable football group buys Ranbir Kapoor's Mumbai City FC
{'Date': '28 Nov', 'Category': 'business', 'Polarity': 0.8553, 'Sentiment': 'Positive'}

Kajol trolls Ajay after he shares pic with her captioned 'Neend churayi meri'
{'Date': '28 Nov', 'Category': 'entertainment', 'Polarity': 0.5994, 'Sentiment': 'Positive'}

9,000 sq ft stage set up for Uddhav's oath ceremony at Shivaji Park
{'Date': '28 Nov', 'Category': 'business', 'Polarity': 0.25, 'Sentiment': 'Positive'}

Naseem is 16 only, Andy Roberts is not his father:

In [50]:
data_set.head()

,Date,Category,Polarity,Sentiment
0,28 Nov,politics,0.0000,Neutral
1,28 Nov,business,-0.4404,Negative
2,28 Nov,entertainment,-0.8442,Negative
3,28 Nov,business,0.8553,Positive
4,28 Nov,entertainment,0.5994,Positive


In [51]:
data_set.to_csv('data/results.csv', index=False)